In [2]:
import pickle
import time
import math
import random
from collections import deque
import networkx as nx

class BrockFinalSolver:
    def __init__(self, nam_file):
        # 1. Load NAM Data
        with open(nam_file, 'rb') as f:
            self.codes_int = pickle.load(f)
        self.num_nodes = len(self.codes_int) - 1
        
        # 2. Structural Analysis (PageRank)
        print("Analyzing graph structure with PageRank...")
        G = nx.Graph()
        for u in range(1, self.num_nodes + 1):
            for v in range(u + 1, self.num_nodes + 1):
                if ((self.codes_int[u] >> v) & 1) == 0:
                    G.add_edge(u, v)
        
        pr_scores = nx.pagerank(G, alpha=0.85)
        self.nodes_list = list(range(1, self.num_nodes + 1))
        
        # We amplify the PageRank to create a strong search bias
        # This focuses the SA on the most 'promising' structural clusters
        self.weights = [pr_scores.get(i, 0)**3 for i in self.nodes_list]
        print("Search bias initialized.")

    def are_neighbors(self, u: int, v: int) -> bool:
        return ((self.codes_int[u] >> v) & 1) == 0

    def get_node_conflicts(self, node, S):
        return sum(1 for member in S if not self.are_neighbors(node, member))

    def solve_once(self, target, max_steps):
        # SA State: Start with a PageRank-weighted random node
        S = random.choices(self.nodes_list, weights=self.weights, k=1)
        tabu = deque(maxlen=55)
        T = 2.0
        alpha = 0.99995
        current_conflicts = 0
        local_best = 1
        
        for i in range(max_steps):
            # Penalty oscillates to allow the SA to 'vibrate' out of traps
            penalty = 22.0 if (i // 1000) % 2 == 0 else 4.0
            
            new_S = list(S)
            move = random.random()

            # --- CLOSE-OUT SCAN ---
            # If we are one node away, scan EVERY node to finish the clique
            if len(new_S) == target - 1:
                for candidate in range(1, self.num_nodes + 1):
                    if candidate not in new_S and self.get_node_conflicts(candidate, new_S) == 0:
                        new_S.append(candidate)
                        return new_S, len(new_S)

            # --- PROBABILISTIC MOVES ---
            if move < 0.12 and len(new_S) > 1:
                # Remove
                node = random.choice(new_S)
                new_S.remove(node)
                tabu.append(node)
            elif move < 0.55:
                # Add with PageRank Bias
                potential = random.choices(self.nodes_list, weights=self.weights, k=70)
                candidates = [n for n in potential if n not in tabu]
                if candidates:
                    best_cand = min(candidates, key=lambda n: self.get_node_conflicts(n, new_S))
                    if best_cand not in new_S: new_S.append(best_cand)
            else:
                # Swap with PageRank Bias
                if len(new_S) > 1:
                    rem = random.choice(new_S)
                    new_S.remove(rem)
                    tabu.append(rem)
                    potential = random.choices(self.nodes_list, weights=self.weights, k=70)
                    candidates = [n for n in potential if n not in tabu]
                    if candidates:
                        add = min(candidates, key=lambda n: self.get_node_conflicts(n, new_S))
                        if add not in new_S: new_S.append(add)

            # --- SA ENERGY LOGIC ---
            new_conf = sum(self.get_node_conflicts(new_S[j], new_S[j+1:]) for j in range(len(new_S)))
            delta = (new_conf * penalty - len(new_S)) - (current_conflicts * penalty - len(S))

            if delta < 0 or random.random() < math.exp(-delta / T):
                S = new_S
                current_conflicts = new_conf
            
            if current_conflicts == 0 and len(S) > local_best:
                local_best = len(S)
            
            T *= alpha
            if T < 0.02: break

        return None, local_best

    def run(self, target=12):
        print(f"Executing PageRank-Weighted SA for Target {target}...")
        start_time = time.time()
        attempt = 0
        global_best = 0
        
        while True:
            attempt += 1
            clique, size = self.solve_once(target, 40000)
            
            if size > global_best:
                global_best = size
                print(f"\n[!] Global Best Updated: Size {global_best}")
            
            if clique:
                print(f"\nSUCCESS! Found Size {target} in {time.time()-start_time:.2f}s")
                return clique
            else:
                print(f"Attempt {attempt} (Current: {size} | Best: {global_best})", end='\r')

if __name__ == "__main__":
    solver = BrockFinalSolver('brock200_2_nam.data')
    result = solver.run(12)
    print(f"Clique Nodes: {sorted(result)}")

Analyzing graph structure with PageRank...
Search bias initialized.
Executing PageRank-Weighted SA for Target 12...

[!] Global Best Updated: Size 10
Attempt 1 (Current: 10 | Best: 10)
[!] Global Best Updated: Size 12

SUCCESS! Found Size 12 in 21.98s
Clique Nodes: [27, 48, 55, 70, 105, 120, 121, 135, 145, 149, 158, 183]
